<a href="https://colab.research.google.com/github/rkdxodnr1234/MY-study/blob/main/%EC%9D%B8%EA%B3%B5%EC%8B%A0%EA%B2%BD%EB%A7%9D_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 실행마다 동일한 결과를 얻기 위해 케라스에 랜덤 시드를 사용하고 텐서플로 연산을 결정적으로 만듭니다.
import tensorflow as tf

tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

In [3]:
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf

In [ ]:
# train 데이터와 동일한 폴더 경로
train_folder_paths = [
    '/content/drive/MyDrive/fruit/train/Apple Red 1',
    '/content/drive/MyDrive/fruit/train/Banana',
    '/content/drive/MyDrive/fruit/train/Lemon',
    '/content/drive/MyDrive/fruit/train/Peach',
    '/content/drive/MyDrive/fruit/train/Pear',
    '/content/drive/MyDrive/fruit/train/Plum'
]

# test 데이터 폴더 경로
test_folder_paths = [
    '/content/drive/MyDrive/fruit/test/Apple Red 1',
    '/content/drive/MyDrive/fruit/test/Banana',
    '/content/drive/MyDrive/fruit/test/Lemon',
    '/content/drive/MyDrive/fruit/test/Peach',
    '/content/drive/MyDrive/fruit/test/Pear',
    '/content/drive/MyDrive/fruit/test/Plum'
]

# 파일명을 가져오는 함수
def get_filenames(folder_paths):
    filenames = []
    for folder_path in folder_paths:
        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg'):
                filenames.append(os.path.join(folder_path, filename))
    return filenames

# train 데이터 파일명을 가져옵니다.
train_file_list = get_filenames(train_folder_paths)

# test 데이터 파일명을 가져옵니다.
test_file_list = get_filenames(test_folder_paths)

# train 데이터를 처리하여 train_input과 train_target을 만듭니다.
train_input = []
train_target = []

for file in train_file_list:
    image = Image.open(file)
    image = image.resize((28, 28))  # 이미지 크기를 조정
    image_array = np.array(image) / 255.0  # 이미지를 0-1 범위로 스케일링
    train_input.append(image_array)
    target = file.split('/')[-2]
    train_target.append(target)

# test 데이터를 처리하여 test_input과 test_target을 만듭니다.
test_input = []
test_target = []

for file in test_file_list:
    image = Image.open(file)
    image = image.resize((28, 28))  # 이미지 크기를 조정
    image_array = np.array(image) / 255.0  # 이미지를 0-1 범위로 스케일링
    test_input.append(image_array)
    target = file.split('/')[-2]
    test_target.append(target)


In [ ]:
# train_input과 train_target, test_input과 test_target을 numpy 배열로 변환합니다.
train_input = np.array(train_input)
train_target = np.array(train_target)
test_input = np.array(test_input)
test_target = np.array(test_target)

# 클래스 이름을 인덱스로 매핑하기 위한 딕셔너리 생성
class_names = ['Apple Red 1', 'Banana', 'Lemon', 'Peach', 'Pear', 'Plum']
class_dict = {class_name: idx for idx, class_name in enumerate(class_names)}

# 타깃을 인덱스로 변환
train_target = np.array([class_dict[target] for target in train_target])
test_target = np.array([class_dict[target] for target in test_target])

In [ ]:
# 데이터 크기 확인
print("train_input shape:", train_input.shape)
print("train_target shape:", train_target.shape)
print("test_input shape:", test_input.shape)
print("test_target shape:", test_target.shape)

In [ ]:
import matplotlib.pyplot as plt

# 첫 번째 이미지 출력
plt.imshow(train_input[0])
plt.axis('off')  # 축 제거
plt.show()

In [ ]:
# train 데이터를 train/val로 분리
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

# 분리 후 데이터 크기 확인
print("train_scaled shape:", train_scaled.shape)
print("train_target shape:", train_target.shape)
print("val_scaled shape:", val_scaled.shape)
print("val_target shape:", val_target.shape)

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28,3)))
model.add(keras.layers.Dense(100, activation='relu', name='hidden'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

In [ ]:
model.evaluate(val_scaled, val_target)

In [ ]:
# 모델 평가
test_loss, test_acc = model.evaluate(test_input, test_target)
print('\n테스트 정확도:', test_acc)

In [ ]:
print(model.predict_classes(test_input[:5]))

fig, axs = plt.subplots(1, 5, figsize=(10,10))
for i in range(5):
    axs[i].imshow(test_input[i], cmap='gray_r')
    axs[i].axis('off')
plt.show()